# BIP0039 生成确定性密钥的助记码

<https://en.bitcoin.it/wiki/BIP_0039>

**BTC私钥** 实际上就是一个32字节的随机数组，而 **公钥** 是通过将椭圆曲线的 **生成元点** 自加而得到，自加的次数就是这个私钥 ，此时私钥被当作一个大整数。而 **BTC地址** 则是通过把公钥哈希而得到。程序逻辑如下：

In [2]:
import random

# 随机生成一个私钥
private_key = random.randint(0, 2**256 - 1)

# 为了方便，我们直接使用一个固定的私钥
private_key = 0x18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321725
print(f"private_key: 0x{private_key:064x}")

private_key: 0x18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321725


In [3]:
# BTC所选择的椭圆曲线是secp256k1, 其参数如下
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a = 0
b = 7
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8

# 椭圆曲线的方程是 y^2 = x^3 + ax + b

# 椭圆曲线上的加法如下，其中 None 表示无穷远点，即加法的单位元，即 0，p1和p2是椭圆曲线上的点
def secp256k1_add(p1, p2):
    if p1 is None:
        return p2
    if p2 is None:
        return p1
    if p1[0] == p2[0] and p1[1] != p2[1]:
        # 两个点互为相反数，即相加为0
        assert (p1[1] + p2[1] % p) == 0
        return None
    # s是斜率
    if p1[0] == p2[0]:
        s = (3 * p1[0] * p1[0] + a) * pow(2 * p1[1], -1, p)
    else:
        s = (p2[1] - p1[1]) * pow(p2[0] - p1[0], -1, p)
    x = (s * s - p1[0] - p2[0]) % p
    y = ( - (s * (x - p1[0]) + p1[1]) ) % p
    return (x, y)

# 私钥 -> 公钥 (倍点算法)
def priv_key_to_pub_key(priv_key):
    # 生成公钥
    pub_key = None
    base = (Gx, Gy)
    for i in range(256):
        if priv_key & (1 << i):
            pub_key = secp256k1_add(pub_key, base)
        base = secp256k1_add(base, base)
    return pub_key

# 生成公钥
pub_key = priv_key_to_pub_key(private_key)
print(f"pub_key: 0x{pub_key[0]:064x}, 0x{pub_key[1]:064x}")

# 压缩形式的公钥
if pub_key[1] & 1:
    compressed_pub_key = 0x03
else:
    compressed_pub_key = 0x02
compressed_pub_key = compressed_pub_key << 256 | pub_key[0]
print(f"compressed_pub_key: 0x{compressed_pub_key:066x}")


pub_key: 0x50863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b2352, 0x2cd470243453a299fa9e77237716103abc11a1df38855ed6f2ee187e9c582ba6
compressed_pub_key: 0x0250863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b2352


In [14]:
import hashlib

# 生成地址
hashed_pub_key = hashlib.sha256(compressed_pub_key.to_bytes(33, 'big')).digest()
print(f"sha256ed_pub_key: 0x{int.from_bytes(hashed_pub_key, 'big'):064x}")

ripemd160_hasher = hashlib.new('ripemd160')
ripemd160_hasher.update(hashed_pub_key)
ripemd160_pub_key = ripemd160_hasher.digest()
print(f"ripemd160ed_pub_key: 0x{int.from_bytes(ripemd160_pub_key, 'big'):040x}")

ripemd160_pub_key_with_version = int.from_bytes(ripemd160_pub_key, 'big').to_bytes(21, 'big') # 添加版本号
print(f"ripemd160_pub_key_with_version: 0x{int.from_bytes(ripemd160_pub_key_with_version, 'big'):042x}")

hashed_ripemd160_pub_key_with_version = hashlib.sha256(ripemd160_pub_key_with_version).digest()
print(f"sha256ed_ripemd160_pub_key_with_version: 0x{int.from_bytes(hashed_ripemd160_pub_key_with_version, 'big'):064x}")

hashed_twice_ripemd160_pub_key_with_version = hashlib.sha256(hashed_ripemd160_pub_key_with_version).digest()
print(f"sha256ed_twice_ripemd160_pub_key_with_version: 0x{int.from_bytes(hashed_twice_ripemd160_pub_key_with_version, 'big'):064x}")

checksum = hashed_twice_ripemd160_pub_key_with_version[:4]
print(f"checksum: 0x{int.from_bytes(checksum, 'big'):08x}")

address = ripemd160_pub_key_with_version + checksum
print(f"address: 0x{int.from_bytes(address, 'big'):050x}")

# base58编码
import base58
address = base58.b58encode(address)
print(f"base58 address: {address.decode('utf-8')}")

sha256ed_pub_key: 0x0b7c28c9b7290c98d7438e70b3d3f7c848fbd7d1dc194ff83f4f7cc9b1378e98
ripemd160ed_pub_key: 0xf54a5851e9372b87810a8e60cdd2e7cfd80b6e31
ripemd160_pub_key_with_version: 0x00f54a5851e9372b87810a8e60cdd2e7cfd80b6e31
sha256ed_ripemd160_pub_key_with_version: 0xad3c854da227c7e99c4abfad4ea41d71311160df2e415e713318c70d67c6b41c
sha256ed_twice_ripemd160_pub_key_with_version: 0xc7f18fe8fcbed6396741e58ad259b5cb16b7fd7f041904147ba1dcffabf747fd
checksum: 0xc7f18fe8
address: 0x00f54a5851e9372b87810a8e60cdd2e7cfd80b6e31c7f18fe8


ModuleNotFoundError: No module named 'base58'